# MBC 김세윤 영화음악 선곡표에서 가장 많이 나온 영화는...?

- 2019년 10월부터 방송을 시작한 MBC FM 김세윤의 영화음악  
- 선곡표를 기준으로 가장 많이 방송된 영화음악은? 


- 주소 정보  

2019-10-01 (첫방송)
http://miniweb.imbc.com/Music/View?progCode=FM4U000001296&seqID=544

2020-02-16 (가장최근 방송)
http://miniweb.imbc.com/Music/View?progCode=FM4U000001296&seqID=674

In [49]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import itertools  

import requests
from bs4 import BeautifulSoup

In [57]:
def html_parser(url):
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def make_date(soup):
    txt = soup.select('div.view-title.color-line.color-bg2')[0].text
    regex = re.compile(r'\d\d\d\d-\d\d-\d\d')
    dt = regex.search(txt)
    return dt.group()

def make_movie(soup):
    movies = soup.select('tr > td.part.color-txt2.color-bg2')  
    # class css에 공백은 점(.)으로 간단히 처리됨 
    # https://stackoverflow.com/questions/46718366/beautifulsoup-and-class-with-spaces
    movie_list = []
    for movie in movies:
        movie_list.append(movie.text)
    return movie_list
        

data = {}
dates = np.arange(544, 675)
# 날짜별로 순환
for date in dates:
    try:        
        url = 'http://miniweb.imbc.com/Music/View?progCode=FM4U000001296&seqID=' + str(date)
        soup = html_parser(url)
        dt = make_date(soup)
        mov = make_movie(soup)
        data[dt] = mov
    except:
        print(date)

596


In [113]:
# 샘플 조회
[data[x] for x in ['2019-10-10','2019-11-10']]

[['미드90',
  '벌새',
  '원스 어폰 어 타임 인... 할리우드',
  '뷰티풀 보이',
  '언더 더 실버레이크',
  '버터플라이 - 말로 하는 신청곡',
  '미쓰 홍당무 - 하루 한 편, 영화자판기',
  '가장 보통의 연애 (예고편)',
  '미드90'],
 ['세상의 끝까지 21일',
  '그해 여름',
  '스카우트',
  '비포 선라이즈',
  '말할 수 없는 비밀',
  '라이언 일병 구하기',
  '쇼생크 탈출',
  '웜 바디스',
  '문라이즈 킹덤',
  '기생충']]

In [76]:
fm_movies = list(itertools.chain(*data.values()))  # 영화목록만 날짜 구분없이 1차원 벡터 리스트로 변환

### 영화명 후처리
- ' - ' 구분자 기준으로 앞만 가져올 것
- 삭제 : 영화자판기 -, (예고편) >> 가장 보통의 연애 (예고편)',
- 좌우 공백 제거 

In [102]:
stop_txts = ['영화자판기 -', '(예고편)']

result = []
for movie in fm_movies:
    if movie.find('-') >= 0:
        movie = movie.split('-')[0]
    for stop_txt in stop_txts:
        movie.replace(stop_txt, "", 1)
    movie = movie.strip()
    result.append(movie)

In [110]:
ser = pd.Series(result)
pd.DataFrame(ser.value_counts()[:50])

,0
라라랜드,11
후 아 유,8
스타 이즈 본,8
조커,7
기생충,7
보이후드,7
보헤미안 랩소디,6
월터의 상상은 현실이 된다,5
원스,5
비긴 어게인,5


---
## 추후 해 볼 것 : 시계열, 장르/국내-해외 별 등 세부 주제로 확장